In [43]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten,Conv2D,MaxPool2D
from keras.preprocessing.image import ImageDataGenerator

In [44]:
train_path="dog_cat_set/train"
test_path="dog_cat_set/test"
validate_path="dog_cat_set/validate"

**Creating batches for training ,validation and test data**

In [45]:
train_set = ImageDataGenerator().flow_from_directory(train_path,target_size=(224,224),classes=['dog','cat'],batch_size=10)
validation_set = ImageDataGenerator().flow_from_directory(validate_path,target_size=(224,224),classes=['dog','cat'],batch_size=5)
test_set = ImageDataGenerator().flow_from_directory(test_path,target_size=(224,224),classes=['dog','cat'],batch_size=10)

Found 200 images belonging to 2 classes.
Found 50 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [46]:
#getting the indices for one hot encoding
train_set.class_indices

{'dog': 0, 'cat': 1}

<h1>creating a model</h1>

In [47]:
# model=Sequential()

In [48]:
# model.add(Conv2D(3,(3,3),activation="relu",input_shape=(224,224,3)))

In [49]:
# model.add(MaxPool2D(pool_size=(2,2)))

In [50]:
# model.add(Conv2D(3,(3,3),activation="relu"))

In [51]:
# model.add(MaxPool2D(pool_size=(2,2)))

In [52]:
# model.add(Flatten())

In [53]:
# model.add(Dense(2,activation="softmax"))

In [54]:
# model.summary()

In [55]:
# model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])

In [56]:
# model.fit_generator(train_set,steps_per_epoch=20,validation_data=validation_set,validation_steps=10,epochs=10)

**This model is doing preety bad let's train a pretrained imagenet model  for dogs and cats images's**

In [57]:
#importing a vgg16 model
vgg16_model = keras.applications.vgg16.VGG16()

In [58]:
vgg16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [59]:
type(vgg16_model)

keras.engine.training.Model

The type of our model is sequential whereas when we get the pretrained model we get the type as Model. Thus we need to access the model layer by layer and add into our sequential model 

In [60]:
model=Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)

Now the vgg16 model was trained to clssify between 1000 different categories(i.e. prediction layer in vgg16) but we need only two categories i.e dog and cat ,thus we need to pop the last layer from the model and then add the last layer according to our need Or you can just copy upto second last layer i.e.excluding the last layer.

In [61]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

we will be using the previous weights for all the layers thus for training process we are declaring layers upto
fc2 as non-trainable

In [62]:
for layer in model.layers:
    layer.trainable = False

In [63]:
model.add(Dense(2,activation="softmax"))

In [64]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [65]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])

In [66]:
# #image augmentation for more correct results
# train_datagen = ImageDataGenerator(
#         rescale=1./255,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True)

# test_datagen = ImageDataGenerator(rescale=1./255)

# valid_datagen = ImageDataGenerator(rescale=1./255)

# training_set = train_datagen.flow_from_directory(
#         'dog_cat_set/train',
#         target_size=(224,224),
#         batch_size=20)

# test_set = test_datagen.flow_from_directory(
#         'dog_cat_set/test',
#         target_size=(224,224),
#         batch_size=5)

# validation_set = valid_datagen.flow_from_directory(
#         'dog_cat_set/validate',
#         target_size=(224,224),
#         batch_size=10)

# model.fit_generator(
#         training_set,
#         steps_per_epoch=40,
#         epochs=5,
#         validation_data=validation_set,
#         validation_steps=10)

In [67]:
model.fit_generator(train_set,steps_per_epoch=20,validation_data=validation_set,validation_steps=10,epochs=3)

Epoch 1/3
20/20 [==============================] - 164s 8s/step - loss: 0.3896 - acc: 0.8600 - val_loss: 0.1071 - val_acc: 0.9400
Epoch 2/3
20/20 [==============================] - 163s 8s/step - loss: 0.0155 - acc: 1.0000 - val_loss: 0.0988 - val_acc: 0.9400
Epoch 3/3
20/20 [==============================] - 163s 8s/step - loss: 0.0030 - acc: 1.0000 - val_loss: 0.1074 - val_acc: 0.9400


In [68]:
model.save("vgg16_re_dc.h5")

In [69]:
# prediction
test_img,test_label = next(test_set)

In [70]:
test_labels = test_label[:,0]

In [71]:
test_labels

array([0., 1., 1., 0., 1., 1., 0., 0., 1., 0.], dtype=float32)

In [72]:

predictio = model.predict_generator(test_set,steps=1)

In [73]:
from sklearn.metrics import confusion_matrix
import numpy as np

In [74]:
predictio = np.round(predictio[:,0])
predictio
# cm = confusion_matrix(test_labels,np.round(pred[:,0]))

array([0., 1., 0., 0., 1., 1., 0., 0., 1., 0.], dtype=float32)

In [75]:
cm = confusion_matrix(test_labels,predictio)
cm

array([[5, 0],
       [1, 4]])

In [84]:
model.evaluate_generator(generator=validation_set,steps=4)

[0.0005135274896161945, 1.0]